In [ ]:
import numpy as np
import pandas as pd
import json
import os
import re
import spacy
nlp = spacy.load('en_core_web_sm')
import os
import pickle

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Concatenate, Dense, Dropout, MultiHeadAttention, Attention
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
word_embeddings = np.load(( "./weights_matrix_100D.npy"), allow_pickle=True)

In [ ]:
with open('traindata.pkl', 'rb') as handle:
    train_df = pickle.load(handle)

In [ ]:
with open('./data/word_vocab.pkl', "rb") as wv, \
      open('./data/word2index.pkl', "rb") as wi, \
         open('./data/index2word.pkl', "rb") as iw:
         word_vocab = pickle.load(wv)
         word2idx = pickle.load(wi)
         idx2word = pickle.load(iw)

In [ ]:
model = tf.keras.models.load_model('final_model.h5')

In [ ]:
max_contex_length = 200
max_question_length = 30
max_sequence_length =256
batch_size = 16
epochs = 5
learning_rate = 1e-3

In [ ]:
def context_to_ids(text, word2idx = word2idx):
    #converts words to numerical index in the text
    context_tokens = [w.text for w in nlp(text, disable=['parser','tagger','ner'])]
    #context_ids = [word2idx[word] for word in context_tokens]
    context_ids = []
    for word in context_tokens:
      if word in word_vocab:
        context_ids.append(word2idx[word])
      else:
        context_ids.append(0)
    assert len(context_ids) == len(context_tokens)
    return context_ids

In [ ]:
def question_to_ids(text, word2idx= word2idx):
    question_tokens = [w.text for w in nlp(text, disable=['parser','tagger','ner'])]
    #question_ids = [word2idx[word] for word in question_tokens]
    question_ids = []
    for word in question_tokens:
      if word in word_vocab:
        question_ids.append(word2idx[word])
      else:
        question_ids.append(0)
    assert len(question_ids) == len(question_tokens)
    return question_ids

In [ ]:
def preprocess(text):
  text= text.lower()
  # text = text.replace('(&#\w+\s*)',"")     # remove html noise e.g -> &#1334
  # text=text.replace('https?://[A-Za-z0-9./]+','') # remove URLs
  # text=text.replace('[^\w\s]|^\s\d+\s|\s\d+|\d+|\s\d+$','') # remove punctuation except '.'
  # text=text.replace('\n',' ') # remove '\n'
  text = re.sub('[^A-Za-z0-9]+', ' ', text)
  return text

In [ ]:
def live_eval():
  context = input("ENTER CONTEXT")
  ques = input("ENTER QUESTION")
  context = preprocess(context)
  ques=preprocess(ques)
  context_ids=context_to_ids(context)
  question_ids=question_to_ids(ques)
  context_ids_ = tf.keras.preprocessing.sequence.pad_sequences(question_ids, maxlen=max_contex_length,padding='post')
  context_ids_ = context_ids_.reshape(1, len(context_ids))

  question_ids_ = tf.keras.preprocessing.sequence.pad_sequences(question_ids, maxlen=max_question_length,padding='post')
  question_ids_ = question_ids_.reshape(1, len(question_ids))
  start_idx, end_idx = model(context_ids_, question_ids_) 

  print(start_idx, end_idx)
  if start_idx<=end_idx:
    for i in context_ids[start_idx:end_idx+1]:
      print(idx2word[i])
    scontext=context.split()
    print(scontext[start_idx:end_idx+1])
  else:
    print('Cannot Answer')
  return context, context_ids, start_idx, end_idx

In [ ]:
ctx1 ='Construction is the process of constructing a building or infrastructure. Construction differs from manufacturing in that manufacturing typically involves mass production of similar items without a designated purchaser, while construction typically takes place on location for a known client. Construction as an industry comprises six to nine percent of the gross domestic product of developed countries. Construction starts with planning,[citation needed] design, and financing and continues until the project is built and ready for use.'
c1_q2='What is the process of constructing a building or infrastructure'
c1_q1='What percentile of gross domestic product is construction comprised of'